# 03 Transform V2 Processed into Historical Averages

## Imports

In [1]:
import pandas as pd
import re

## Pull in V2 Processed DataFrame

In [2]:
df = pd.read_csv('../../02_Data/02_Processed_Data/V2_Processed.csv', index_col=0)

In [3]:
df.shape

(4774, 225)

In [4]:
def calculate_expanding_mean(df):
    # Find the unique ID for every fighter
    unique_ids = df.f1_fighterid.unique()
    
    cols_to_calc_em = df.select_dtypes(exclude='object').columns
    cols_to_calc_em = [col for col in cols_to_calc_em if col not in ['eventid','fightid','currentrnd','maxrnds',
                                                                     'f1_fighterid','f2_fighterid']]
    
    for count, unique_id in enumerate(unique_ids):
        new_df = df[(df.f1_fighterid == unique_id)]
        if count%50==0:
            print(f"{count} in {len(unique_ids)}")
        
        
        # Calculate Expanding Mean
        df_stub = new_df.sort_values('date')[['eventid','fightid', 'f1_fighterid', 'date']].copy()
        df_em_part = new_df.sort_values('date').expanding().mean().shift()[cols_to_calc_em]
        df_em_part.columns = [col +'_avg' for col in df_em_part.columns]
        df_em = pd.concat([df_stub,df_em_part], axis=1).copy()        
        df_em = df_em.iloc[1:, :]
        
        try:
            df_ems = pd.concat([df_ems, df_em], axis=0)

        except NameError:
            df_ems = df_em.copy() # If the dataframe doesn't exist yet, then copy it from df_em
            
        
        
    return df_ems

In [5]:
historical_avg = calculate_expanding_mean(df)

0 in 1091
50 in 1091
100 in 1091
150 in 1091
200 in 1091
250 in 1091
300 in 1091
350 in 1091
400 in 1091
450 in 1091
500 in 1091
550 in 1091
600 in 1091
650 in 1091
700 in 1091
750 in 1091
800 in 1091
850 in 1091
900 in 1091
950 in 1091
1000 in 1091
1050 in 1091


In [6]:
historical_avg.shape

(3683, 219)

## Export this and just first join back

In [7]:
historical_avg.to_csv('../../02_Data/02_Processed_Data/historical_avgs.csv')